<a href="https://colab.research.google.com/github/mco-gh/pylearn/blob/master/notebooks/9_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook 9 - Sample Project

**Make a copy of this notebook by selecting File->Save a copy in Drive from the menu bar above.**

Things you'll learn in this lesson:
- how to formulate and carry out a Python project

[Previous Lesson](https://pylearn.io/lessons/8-Files/)

## Quizaic

[Previous Lesson](https://pylearn.io/lessons/8-Files/)